In [ ]:
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

warnings.filterwarnings("ignore", category=FutureWarning)

# Create SparkSession
spark = SparkSession.builder.appName("DeltaSession") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .getOrCreate()

In [ ]:
dt = DeltaTable.forPath(spark, 'extract/01delta')

dt.toDF().toPandas()

In [ ]:
dt.history().toPandas()

## versionAsOf

In [ ]:
!pwd

In [ ]:
spark.sql("""
SELECT * FROM delta.`/home/jovyan/delta/extract/01delta` 
VERSION AS OF 0
""").toPandas()

In [ ]:
(spark.read
     .format("delta")
     .option("versionAsOf", 0)
     .load('extract/01delta')
).toPandas()

In [ ]:
(spark.read
     .format("delta")
     .option("versionAsOf", 4)
     .load('extract/01delta')
).toPandas()

## timestampAsOf

In [ ]:
spark.sql("""
SELECT * FROM delta.`/home/jovyan/delta/extract/01delta` 
TIMESTAMP AS OF '2023-05-02 21:19:30.502'
""").toPandas()

In [ ]:
(spark.read
 .format("delta")
 .option("timestampAsOf", "2023-05-02 21:19:30.502")
 .load('extract/01delta')
).toPandas()

## restoreToVersion

In [ ]:
dt = DeltaTable.forPath(spark, 'extract/01delta')

dt.toDF().toPandas()

In [ ]:
# sql
spark.sql("""
RESTORE TABLE delta.`/home/jovyan/delta/extract/01delta`
TO VERSION AS OF 1""").toPandas()

In [ ]:
# python
dt.restoreToVersion(1).toPandas()

In [ ]:
# show delta table
DeltaTable.forPath(spark, 'extract/01delta').toDF().toPandas()

## restoreToTimestamp

In [ ]:
# restore sql
spark.sql("""
RESTORE TABLE delta.`/home/jovyan/delta/extract/01delta`
TO TIMESTAMP AS OF '2023-05-02 21:19:30.502'
""").toPandas()

In [ ]:
spark.sql("""
DESCRIBE HISTORY delta.`/home/jovyan/delta/extract/01delta` LIMIT 5
""").toPandas()

In [ ]:
dt = DeltaTable.forPath(spark, 'extract/01delta')

dt.toDF().toPandas()

In [ ]:
dt.restoreToTimestamp("2023-05-02 21:19:30.502").toPandas()

In [ ]:
dt.history().toPandas()

In [ ]:
# show delta table
DeltaTable.forPath(spark, 'extract/01delta').toDF().toPandas()

## vacuum

In [ ]:
spark.sql("SHOW TBLPROPERTIES delta.`/home/jovyan/delta/extract/01delta`").toPandas()

In [ ]:
spark.sql("""
ALTER TABLE delta.`/home/jovyan/delta/extract/01delta`
SET TBLPROPERTIES(
 delta.logRetentionDuration = "interval 30 days",
 delta.deletedFileRetentionDuration = "interval 7 days"
)
""")

In [ ]:
spark.sql("SHOW TBLPROPERTIES delta.`/home/jovyan/delta/extract/01delta`").toPandas()

In [ ]:
dt.history(3).toPandas()

In [ ]:
# sql
spark.sql("VACUUM delta.`/home/jovyan/delta/extract/01delta` RETAIN 3 HOURS")

### spark.databricks.delta.retentionDurationCheck.enabled

In [ ]:
# sql
# spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")

# python
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [ ]:
# sql
spark.sql("VACUUM delta.`/home/jovyan/delta/extract/01delta` RETAIN 3 HOURS").toPandas()

In [ ]:
spark.sql("""
DESCRIBE HISTORY delta.`/home/jovyan/delta/extract/01delta` LIMIT 5
""").toPandas()

In [ ]:
dt = DeltaTable.forPath(spark, 'extract/01delta')

dt.toDF().toPandas()

In [ ]:
# python
dt.vacuum(retentionHours=0)

In [ ]:
dt.history().toPandas()

In [ ]:
(spark.read
     .format("delta")
     .option("versionAsOf", 0)
     .load('extract/01delta')
).toPandas()

In [ ]:
dt.restoreToVersion(0).toPandas()